# Exploration of the stochastic BusSim model

In [ ]:
from BusSim_stochastic import run_model as run_model_s
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mogp_emulator

# Increase default figure size
plt.rcParams["figure.figsize"] = [12, 6]
plt.rcParams["font.size"] = 16

## Setup

First we need a function to run the model; this is pretty much the same as the deterministic version in the Exploration.ipynb file.

In [ ]:
def run_stochastic_model(ArrivalRate=None, TrafficSpeed=14, bus_ids_out=[0, 1, 2], maxDemand=2, DEBUG=False):
    
    NumberOfStop = 20
    minDemand = 0.5
     
    # Initialise the remaining model parameters
    model_params = {
        "dt": 10,
        "minDemand": minDemand,        
        "NumberOfStop": NumberOfStop,
        "LengthBetweenStop": 2000,
        "EndTime": 6000,
        "Headway": 5 * 60,
        "BurnIn": 1 * 60,
        "AlightTime": 1,
        "BoardTime": 3,
        "StoppingTime": 3,
        "BusAcceleration": 3
    }
    
    if ArrivalRate is None:
        ArrivalRate = np.random.uniform(minDemand / 60, maxDemand / 60, NumberOfStop) 
    else:
        ArrivalRate = ArrivalRate * np.ones(NumberOfStop)
        
    DepartureRate = np.sort(np.random.uniform(0.05, 0.5, NumberOfStop)) # Sorted as more passengers get off near the end of the route

    model, model_params, ArrivalData, StateData, GroundTruth, GPSData = \
        run_model_s(model_params, TrafficSpeed, ArrivalRate, DepartureRate, False, False, False, True)
            
    bus_pos = np.array([bus.trajectory for bus in model.buses])    
    time = np.arange(0, model.EndTime, model.dt)
    total_distance = model_params["NumberOfStop"] * model_params["LengthBetweenStop"]

    bus_t_end = [time[np.argmax(bp>=total_distance)] for bp in bus_pos]
    bus_end_times = [bus_t_end[id] for id in bus_ids_out]
    
    if DEBUG:
        print(bus_t_end)
    
    return bus_end_times, GPSData

## Initial look at variation

We'll start by taking a look at the variability in bus arrival times for two different values of traffic speed.

In [ ]:
%%capture
bus_ids = range(8)
repeats = 10

md = 6
ts = 14
ts_fast = 28

end_times = np.zeros((len(bus_ids), repeats))
gps_locs = []
for r in range(repeats):
    t, g = run_stochastic_model(TrafficSpeed=ts, bus_ids_out=bus_ids, maxDemand=md)
    end_times[:,r] = t
    gps_locs.append(g)
    
end_times_fast = np.zeros((len(bus_ids), repeats))
gps_locs_fast = []
for r in range(repeats):
    t, g = run_stochastic_model(TrafficSpeed=ts_fast, bus_ids_out=bus_ids, maxDemand=md)
    end_times_fast[:,r] = t
    gps_locs_fast.append(g)

First let's check out what happens with the default speed of 14.
Note that some of the buses that depart later do not reach the final destination by the last time-step; those arrival time values are therefore shown as 0.

In [ ]:
plt.plot(bus_ids, end_times);
plt.ylim((3000, 7000));
plt.xlabel("Bus ID");
plt.ylabel("Arrival time at final stop");

We'll also compare those buses against others travelling at a much faster speed.
We have dropped colours for clarity here; slow buses are shown in black and fast ones in blue.

In [ ]:
plt.plot(bus_ids, end_times, color="k");
plt.plot(bus_ids, end_times_fast, color="b");
plt.ylim((2000, 7000));
plt.xlabel("Bus ID");
plt.ylabel("Arrival time at final stop");

## Using a GP to emulate bus arrival times with variable traffic speed

Before we start, what might we expect to see as an output from the GP this time?

In the previous notebook, we used a GP to predict final arrival times for different traffic speeds.
That GP generally showed very low uncertainty, apart from at the limits of the traffic speed parameter space and sometimes also for Bus 0, which was often overtaken by the bus that followed it when the traffic speeds were higher.

### Generate training data for the GP

We'll consider the same set-up as last time: we will vary traffic speed between 15 and 50, and use four training points within that parameter space.
The difference this time is that we will run repeated simulations at each of those training points, so that we have 5 repeats per value of traffic speed.

In [ ]:
ts_bus_ids = [0, 1, 2, 3, 6, 9]

lhd = mogp_emulator.LatinHypercubeDesign([(15.0, 50.0)])

n_simulations = 20
n_repeats = 5
n_sims = n_simulations * n_repeats
lhd_traffic_speeds = lhd.sample(n_simulations)
print("Using these traffic speed values for generating training data:")
print(lhd_traffic_speeds)
lhd_traffic_speeds = np.repeat(lhd_traffic_speeds, n_repeats)

In [ ]:
%%capture

ts_simulation_output = np.zeros((len(ts_bus_ids), n_sims))

for ind, ts in enumerate(lhd_traffic_speeds):
    sim_out, _ = run_stochastic_model(TrafficSpeed=ts, bus_ids_out=ts_bus_ids)
    ts_simulation_output[:,ind] = np.array(sim_out)

### Fit the GP

We'll actually fit two different models here to try out the effects of changing the kernel.

In the first model, we use the standard squared exponential kernelm while in the second, we use the Matern 5-2 kernel.

The Matern kernel produces functions that are less smooth than the (default) squared exponential - it may be better at capturing the variability in arrival times.

In [ ]:
ts_gp_se = mogp_emulator.MultiOutputGP(lhd_traffic_speeds, ts_simulation_output, kernel=mogp_emulator.Kernel.SquaredExponential(), nugget="adaptive")
ts_gp_se = mogp_emulator.fit_GP_MAP(ts_gp_se)

ts_gp_se_nfit = mogp_emulator.MultiOutputGP(lhd_traffic_speeds, ts_simulation_output, kernel=mogp_emulator.Kernel.SquaredExponential(), nugget="fit")
ts_gp_se_nfit = mogp_emulator.fit_GP_MAP(ts_gp_se_nfit)

ts_gp_m52 = mogp_emulator.MultiOutputGP(lhd_traffic_speeds, ts_simulation_output, kernel=mogp_emulator.Kernel.Matern52(), nugget="adaptive")
ts_gp_m52 = mogp_emulator.fit_GP_MAP(ts_gp_m52)

ts_gp_m52_nfit = mogp_emulator.MultiOutputGP(lhd_traffic_speeds, ts_simulation_output, kernel=mogp_emulator.Kernel.Matern52(), nugget="fit")
ts_gp_m52_nfit = mogp_emulator.fit_GP_MAP(ts_gp_m52_nfit)

### Try out the GP

Note the warning above: `Matrix not positive definite, skipping this iteration`

This is generated because there are different values of the final arrival time for the same input value of traffic speed.

To check in `mogp_emulator`: how does the nugget link in here? A nugget can be added for computational reasons - it is a small term added to the diagonal of the covariance matrix. However, is can also be interpreted as a small (consistent?) amount of stochastic noise

In [ ]:
%%capture
n_validation = 50

validation_traffic_speeds = lhd.sample(n_validation)

nugget_setting = False

predicted_ts_se = ts_gp_se.predict(validation_traffic_speeds, include_nugget=nugget_setting)
predicted_ts_se_nfit = ts_gp_se_nfit.predict(validation_traffic_speeds, include_nugget=nugget_setting)
predicted_ts_m52 = ts_gp_m52.predict(validation_traffic_speeds, include_nugget=nugget_setting)
predicted_ts_m52_nfit = ts_gp_m52_nfit.predict(validation_traffic_speeds, include_nugget=nugget_setting)


actual_ts = np.zeros((len(ts_bus_ids), n_validation))
for ind, ts in enumerate(validation_traffic_speeds):
    sim_out, _ = run_stochastic_model(TrafficSpeed=ts[0], bus_ids_out=ts_bus_ids)
    actual_ts[:,ind] = np.array(sim_out)

Comments so far
* MOGP lets us create a GP despite multiple outputs for consistent inputs
* Huge uncertainty for buses after the first with comparable training points to deterministic model, despite very consistent outputs
* Adding more repeats does bring down uncertainty, but overall for one bus rather than in specific regions

In [ ]:
# Define a function for plotting data used to train a gp, the simulated outputs and emulated outputs
def plot_gp_train_and_pred(train_inputs, test_inputs, train_outputs, simulated_outputs, predicted_outputs, bus_ids, plt_bus_index=None):
    
    # Set the markers used in the plot
    plt_mk = { "train": ".",      # Used to train the GP
               "pred": "x",    # Predicted by the GP with squared exponential kernel
               "sim": "+" }       # Same TS as "pred", full simulation
    plt_mk_legend = [matplotlib.lines.Line2D([0], [0], marker=plt_mk["train"], color="w", label="Training",
                     markerfacecolor="k", markeredgecolor="k"), 
                     matplotlib.lines.Line2D([0], [0], marker=plt_mk["pred"], color="w", label="Predicted (GP)",
                     markerfacecolor="k", markeredgecolor="k"),
                     matplotlib.lines.Line2D([0], [0], marker=plt_mk["sim"], color="w", label="Simulated",
                     markerfacecolor="k", markeredgecolor="k")]
    
    legend_entries = []

    # Easier to look at one bus at a time
    if plt_bus_index:
        plt.title("Bus {}".format(bus_ids[plt_bus_index]))

    for ind, (a, p, u) in enumerate(zip(simulated_outputs, predicted_outputs.mean, predicted_outputs.unc)):

        if plt_bus_index is None or ind == plt_bus_index:
            plt.plot(test_inputs, a, marker=plt_mk["sim"], linestyle="", color=plt.cm.Set1(ind))
            plt.plot(test_inputs, p, marker=plt_mk["pred"], linestyle="", color=plt.cm.Set1(ind))
            plt.plot(train_inputs, train_outputs[ind,], marker=plt_mk["train"], linestyle="", color=plt.cm.Set1(ind))

            legend_entries.append(matplotlib.patches.Patch(facecolor=plt.cm.Set1(ind), edgecolor=None, label="Bus {}".format(bus_ids[ind])))

            o = np.argsort(validation_traffic_speeds, axis=None)
            plt.fill_between(validation_traffic_speeds.flatten()[o], p[o]-u[o], p[o]+u[o],
                             color=plt.cm.Set1(ind), alpha=0.2, linewidth=0)

    plt.xlabel("Traffic speed")
    plt.ylabel("Time at final\ndestination")
    plt.legend(handles=legend_entries+plt_mk_legend, fontsize="small", loc="center right", bbox_to_anchor=(1.3,0.5))
    plt.ylim((0, 8000))

In [ ]:
plt_bus_index = 4

fig, axs = plt.subplots(2,2,figsize=(25,15))
fig.subplots_adjust(wspace=0.5, hspace=0.3)

plt.sca(axs[0,0])
plot_gp_train_and_pred(lhd_traffic_speeds, validation_traffic_speeds, 
                      ts_simulation_output, actual_ts, predicted_ts_se, ts_bus_ids, plt_bus_index);
plt.title("Squared exponential kernel (adaptive)");

plt.sca(axs[1,0])
plot_gp_train_and_pred(lhd_traffic_speeds, validation_traffic_speeds, 
                      ts_simulation_output, actual_ts, predicted_ts_se_nfit, ts_bus_ids, plt_bus_index);
plt.title("Squared exponential kernel (fit)");

plt.sca(axs[0,1])
plot_gp_train_and_pred(lhd_traffic_speeds, validation_traffic_speeds, 
                      ts_simulation_output, actual_ts, predicted_ts_m52, ts_bus_ids, plt_bus_index);
plt.title("Matern 5-2 kernel (adaptive)");

plt.sca(axs[1,1])
plot_gp_train_and_pred(lhd_traffic_speeds, validation_traffic_speeds, 
                      ts_simulation_output, actual_ts, predicted_ts_m52_nfit, ts_bus_ids, plt_bus_index);
plt.title("Matern 5-2 kernel (fit)");

### Brief notes on alternative kernels

- For 20 repeats, 10 sims: SE kernel seems to pick up gaps in the training data better that the M52 does - look at Bus 9 (index 5). However, still have wide bounds on other buses and it isn't clear why.
- 20 sims, 10 repeats: see M52 more (too) responsive to gaps in data...

### Get nugget parameter (adaptive or fit)

In [ ]:
print("Traffic speed emulator with squared exponential kernel and adaptive nugget")
for bus, em in zip(ts_bus_ids, ts_gp_se.emulators):
    print("Bus {}\t{}\t({})".format(bus, em.nugget, em.nugget_type))

print("\nTraffic speed emulator with squared exponential kernel and fit nugget")
for bus, em in zip(ts_bus_ids, ts_gp_se_nfit.emulators):
    print("Bus {}\t{}\t({})".format(bus, em.nugget, em.nugget_type))
    
print("\nTraffic speed emulator with Matern 5-2 kernel and adaptive nugget")
for bus, em in zip(ts_bus_ids, ts_gp_m52.emulators):
    print("Bus {}\t{}\t({})".format(bus, em.nugget, em.nugget_type))
    
print("\nTraffic speed emulator with Matern 5-2 kernel and fit nugget")
for bus, em in zip(ts_bus_ids, ts_gp_m52_nfit.emulators):
    print("Bus {}\t{}\t({})".format(bus, em.nugget, em.nugget_type))

To do:
  - How accurate is the uncertainty estimate?
  - Take multiple draws at a given point; what is the distribution? (We plot mean +/- uncertainty, so lines are smooth)
  - Does the simulated data match up with out uncertainty estimates around the emulator?

## Accuracy of the uncertainty estimate

With the `include_nugget` option turned off, we are getting much more reasonable results from the above GPs.

Now let's get try to understand how well the uncertainty in the model is quantified, by adding further simulation runs to the above plot.

In [ ]:
%%capture
n_validation_repeats = 10

further_ts = np.zeros((len(ts_bus_ids), n_validation, n_validation_repeats))
for ind, ts in enumerate(validation_traffic_speeds):
    for ind_rep in range(n_validation_repeats):
        sim_out, _ = run_stochastic_model(TrafficSpeed=ts[0], bus_ids_out=ts_bus_ids)
        further_ts[:,ind,ind_rep] = np.array(sim_out)  # dim: bus_ind, validation ts, repeat

In [ ]:
# Define a function for plotting data used to train a gp, the simulated outputs and emulated outputs
def plot_gp_and_sims(inputs, simulated_outputs, predicted_outputs, bus_ids, plt_bus_index=None):
    
    # # Set the markers used in the plot
    # plt_mk = { "train": ".",      # Used to train the GP
    #            "pred": "x",    # Predicted by the GP with squared exponential kernel
    #            "sim": "+" }       # Same TS as "pred", full simulation
    # plt_mk_legend = [matplotlib.lines.Line2D([0], [0], marker=plt_mk["train"], color="w", label="Training",
    #                  markerfacecolor="k", markeredgecolor="k"), 
    #                  matplotlib.lines.Line2D([0], [0], marker=plt_mk["pred"], color="w", label="Predicted (GP)",
    #                  markerfacecolor="k", markeredgecolor="k"),
    #                  matplotlib.lines.Line2D([0], [0], marker=plt_mk["sim"], color="w", label="Simulated",
    #                  markerfacecolor="k", markeredgecolor="k")]
    
    legend_entries = []
    
    input_order = np.argsort(inputs, axis=None)

    # Easier to look at one bus at a time
    if plt_bus_index:
        plt.title("Bus {}".format(bus_ids[plt_bus_index]))

    for ind, (a, p, u) in enumerate(zip(simulated_outputs, predicted_outputs.mean, predicted_outputs.unc)):

        if plt_bus_index is None or ind == plt_bus_index:
            plt.plot(inputs[input_order], p[input_order], marker="", linestyle="-", color=plt.cm.Set1(ind))

            # legend_entries.append(matplotlib.patches.Patch(facecolor=plt.cm.Set1(ind), edgecolor=None, label="Bus {}".format(bus_ids[ind])))
            
            plt.fill_between(inputs.flatten()[input_order], p[input_order]-u[input_order], p[input_order]+u[input_order],
                             color=plt.cm.Set1(ind), alpha=0.2, linewidth=0)
            
            for inp, sim in zip(inputs, simulated_outputs[plt_bus_index]):
                plt.scatter(inp*np.ones((len(sim),1)), sim, marker=".", color="k", s=5)
                sim_std = np.std(sim)
                sim_mean = np.mean(sim)
                plt.scatter([inp, inp], sim_mean+[-2*sim_std, 2*sim_std], marker="_", color="k", s=50)

    plt.xlabel("Traffic speed")
    plt.ylabel("Time at final\ndestination")
    # plt.legend(handles=legend_entries+plt_mk_legend, fontsize="small", loc="center right", bbox_to_anchor=(1.3,0.5))
    plt.ylim((0, 8000))

In [ ]:
plt_bus_index = 3
plot_gp_and_sims(validation_traffic_speeds, further_ts, predicted_ts_se_nfit, ts_bus_ids, plt_bus_index)
plt.title("Squared exponential kernel (fit)");

Notes:
  - 2 std dev variation in simulator is reasonably well-captured by GP, apart from at edges
  - Nugget term is *very* large for "fit" nugget size on GPs for later buses.
    - Check in code - could this be because the nugget is optimised for all separate GPs, and the final ones in this test are outliers in that they have much higher arrival times (we look at [0, 1, 2, 3, *gap* 6, *gap* 9])?
    - No, tried with buses [8, 9, 10] and saw the same high uncertainty bounds as before. Also nugget terms differed greatly between fit and adaptive, surprisingly being much larger with the fit method.
    - Check in MOGP - when does "fit" decide the optimal value is reached? Is it overfitting to the exact values at the expense of getting a sensible value for the uncertainty?
    
    